### 🔧 环境配置和检查

#### 概述

本教程需要特定的环境配置以确保最佳学习体验。以下配置将帮助您：

- 使用统一的conda环境：激活统一的学习环境
- 通过国内镜像源快速安装依赖：配置pip使用清华镜像源
- 加速模型下载：设置HuggingFace镜像代理
- 检查系统配置：检查硬件和软件配置

#### 配置

- **所需环境及其依赖已经部署好**
- 在`Notebook`右上角选择`jupyter内核`为`python(flyai_agent_in_action)`，即可执行下方代码

In [ ]:
%%script bash

# 1. 激活 conda 环境 (仅对当前单元格有效)
eval "$(conda shell.bash hook)"
conda activate flyai_agent_in_action

echo "========================================="
echo "== Conda 环境检查报告 (仅针对当前 Bash 子进程) =="
echo "========================================="

# 2. 检查当前激活的环境
CURRENT_ENV_NAME=$(basename $CONDA_PREFIX)

if [ "$CURRENT_ENV_NAME" = "flyai_agent_in_action" ]; then
    echo "✅ 当前单元格已成功激活到 flyai_agent_in_action 环境。"
    echo "✅ 正在使用的环境路径: $CONDA_PREFIX"
    echo ""
    echo "💡 提示: 后续的 Python 单元格将使用 Notebook 当前选择的 Jupyter 内核。"
    echo "   如果需要后续单元格也使用此环境，请执行以下操作:"
    echo "   1. 检查 Notebook 右上角是否已选择 'python(flyai_agent_in_action)'。"
else
    echo "❌ 激活失败或环境名称不匹配。当前环境: $CURRENT_ENV_NAME"
    echo ""
    echo "⚠️ 严重提示: 建议将 Notebook 的 Jupyter **内核 (Kernel)** 切换为 'python(flyai_agent_in_action)'。"
    echo "   (通常位于 Notebook 右上角或 '内核' 菜单中)"
    echo ""
    echo "📚 备用方法 (不推荐): 如果无法切换内核，则必须在**每个**代码单元格的头部重复以下命令:"
    echo ""
    echo "%%script bash"
    echo "# 必须在每个单元格都执行"
    echo "eval \"\$(conda shell.bash hook)\""
    echo "conda activate flyai_agent_in_action"
fi

echo "=========================================" 

In [ ]:
# 2. 设置pip 为清华源
%pip config set global.index-url https://pypi.tuna.tsinghua.edu.cn/simple
%pip config list


In [ ]:
# 3. 设置HuggingFace代理
%env HF_ENDPOINT=https://hf-mirror.com
# 验证：使用shell命令检查
!echo $HF_ENDPOINT

In [ ]:
# 🔍 环境信息检查脚本
#
# 本脚本的作用：
# 1. 安装 pandas 库用于数据表格展示
# 2. 检查系统的各项配置信息
# 3. 生成详细的环境报告表格
#
# 对于初学者来说，这个步骤帮助您：
# - 了解当前运行环境的硬件配置
# - 确认是否满足模型运行的最低要求
# - 学习如何通过代码获取系统信息

# 安装 pandas 库 - 用于创建和展示数据表格
# pandas 是 Python 中最流行的数据处理和分析库
%pip install pandas==2.2.2 tabulate==0.9.0

import platform # 导入 platform 模块以获取系统信息
import os # 导入 os 模块以与操作系统交互
import subprocess # 导入 subprocess 模块以运行外部命令
import pandas as pd # 导入 pandas 模块，通常用于数据处理，这里用于创建表格
import shutil # 导入 shutil 模块以获取磁盘空间信息

# 获取 CPU 信息的函数，包括核心数量
def get_cpu_info():
    cpu_info = "" # 初始化 CPU 信息字符串
    physical_cores = "N/A"
    logical_cores = "N/A"

    if platform.system() == "Windows": # 如果是 Windows 系统
        cpu_info = platform.processor() # 使用 platform.processor() 获取 CPU 信息
        try:
            # 获取 Windows 上的核心数量 (需要 WMI)
            import wmi
            c = wmi.WMI()
            for proc in c.Win32_Processor():
                physical_cores = proc.NumberOfCores
                logical_cores = proc.NumberOfLogicalProcessors
        except:
            pass # 如果 WMI 不可用，忽略错误

    elif platform.system() == "Darwin": # 如果是 macOS 系统
        # 在 macOS 上使用 sysctl 命令获取 CPU 信息和核心数量
        os.environ['PATH'] = os.environ['PATH'] + os.pathsep + '/usr/sbin' # 更新 PATH 环境变量
        try:
            process_brand = subprocess.Popen(['sysctl', "machdep.cpu.brand_string"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout_brand, stderr_brand = process_brand.communicate()
            cpu_info = stdout_brand.decode().split(': ')[1].strip() if stdout_brand else "Could not retrieve CPU info"

            process_physical = subprocess.Popen(['sysctl', "hw.physicalcpu"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout_physical, stderr_physical = process_physical.communicate()
            physical_cores = stdout_physical.decode().split(': ')[1].strip() if stdout_physical else "N/A"

            process_logical = subprocess.Popen(['sysctl', "hw.logicalcpu"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout_logical, stderr_logical = process_logical.communicate()
            logical_cores = stdout_logical.decode().split(': ')[1].strip() if stdout_logical else "N/A"

        except:
            cpu_info = "Could not retrieve CPU info"
            physical_cores = "N/A"
            logical_cores = "N/A"

    else:  # Linux 系统
        try:
            # 在 Linux 上读取 /proc/cpuinfo 文件获取 CPU 信息和核心数量
            with open('/proc/cpuinfo') as f:
                physical_cores_count = 0
                logical_cores_count = 0
                cpu_info_lines = []
                for line in f:
                    if line.startswith('model name'): # 查找以 'model name'开头的行
                        if not cpu_info: # 只获取第一个 model name
                            cpu_info = line.split(': ')[1].strip()
                    elif line.startswith('cpu cores'): # 查找以 'cpu cores' 开头的行
                        physical_cores_count = int(line.split(': ')[1].strip())
                    elif line.startswith('processor'): # 查找以 'processor' 开头的行
                        logical_cores_count += 1
                physical_cores = str(physical_cores_count) if physical_cores_count > 0 else "N/A"
                logical_cores = str(logical_cores_count) if logical_cores_count > 0 else "N/A"
                if not cpu_info:
                     cpu_info = "Could not retrieve CPU info"

        except:
            cpu_info = "Could not retrieve CPU info"
            physical_cores = "N/A"
            logical_cores = "N/A"

    return f"{cpu_info} ({physical_cores} physical cores, {logical_cores} logical cores)" # 返回 CPU 信息和核心数量


# 获取内存信息的函数
def get_memory_info():
    mem_info = "" # 初始化内存信息字符串
    if platform.system() == "Windows":
        # 在 Windows 上不容易通过标准库获取，需要外部库或 PowerShell
        mem_info = "Requires external tools on Windows" # 设置提示信息
    elif platform.system() == "Darwin": # 如果是 macOS 系统
        # 在 macOS 上使用 sysctl 命令获取内存大小
        process = subprocess.Popen(['sysctl', "hw.memsize"], stdout=subprocess.PIPE, stderr=subprocess.PIPE) # 运行 sysctl 命令
        stdout, stderr = process.communicate() # 获取标准输出和标准错误
        mem_bytes = int(stdout.decode().split(': ')[1].strip()) # 解析输出，获取内存大小（字节）
        mem_gb = mem_bytes / (1024**3) # 转换为 GB
        mem_info = f"{mem_gb:.2f} GB" # 格式化输出
    else:  # Linux 系统
        try:
            # 在 Linux 上读取 /proc/meminfo 文件获取内存信息
            with open('/proc/meminfo') as f:
                total_mem_kb = 0
                available_mem_kb = 0
                for line in f:
                    if line.startswith('MemTotal'): # 查找以 'MemTotal' 开头的行
                        total_mem_kb = int(line.split(':')[1].strip().split()[0]) # 解析行，获取总内存（KB）
                    elif line.startswith('MemAvailable'): # 查找以 'MemAvailable' 开头的行
                         available_mem_kb = int(line.split(':')[1].strip().split()[0]) # 解析行，获取可用内存（KB）

                if total_mem_kb > 0:
                    total_mem_gb = total_mem_kb / (1024**2) # 转换为 GB
                    mem_info = f"{total_mem_gb:.2f} GB" # 格式化输出总内存
                    if available_mem_kb > 0:
                        available_mem_gb = available_mem_kb / (1024**2)
                        mem_info += f" (Available: {available_mem_gb:.2f} GB)" # 添加可用内存信息
                else:
                     mem_info = "Could not retrieve memory info" # 如果读取文件出错，设置错误信息

        except:
            mem_info = "Could not retrieve memory info" # 如果读取文件出错，设置错误信息
    return mem_info # 返回内存信息

# 获取 GPU 信息的函数，包括显存
def get_gpu_info():
    try:
        # 尝试使用 nvidia-smi 获取 NVIDIA GPU 信息和显存
        result = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total', '--format=csv,noheader'], capture_output=True, text=True)
        if result.returncode == 0: # 如果命令成功执行
            gpu_lines = result.stdout.strip().split('\n') # 解析输出，获取 GPU 名称和显存
            gpu_info_list = []
            for line in gpu_lines:
                name, memory = line.split(', ')
                gpu_info_list.append(f"{name} ({memory})") # 格式化 GPU 信息
            return ", ".join(gpu_info_list) if gpu_info_list else "NVIDIA GPU found, but info not listed" # 返回 GPU 信息或提示信息
        else:
             # 尝试使用 lshw 获取其他 GPU 信息 (需要安装 lshw)
            try:
                result_lshw = subprocess.run(['lshw', '-C', 'display'], capture_output=True, text=True)
                if result_lshw.returncode == 0: # 如果命令成功执行
                     # 简单解析输出中的 product 名称和显存
                    gpu_info_lines = []
                    current_gpu = {}
                    for line in result_lshw.stdout.splitlines():
                        if 'product:' in line:
                             if current_gpu:
                                 gpu_info_lines.append(f"{current_gpu.get('product', 'GPU')} ({current_gpu.get('memory', 'N/A')})")
                             current_gpu = {'product': line.split('product:')[1].strip()}
                        elif 'size:' in line and 'memory' in line:
                             current_gpu['memory'] = line.split('size:')[1].strip()

                    if current_gpu: # 添加最后一个 GPU 的信息
                        gpu_info_lines.append(f"{current_gpu.get('product', 'GPU')} ({current_gpu.get('memory', 'N/A')})")

                    return ", ".join(gpu_info_lines) if gpu_info_lines else "GPU found (via lshw), but info not parsed" # 如果找到 GPU 但信息无法解析，设置提示信息
                else:
                    return "No GPU found (checked nvidia-smi and lshw)" # 如果两个命令都找不到 GPU，设置提示信息
            except FileNotFoundError:
                 return "No GPU found (checked nvidia-smi, lshw not found)" # 如果找不到 lshw 命令，设置提示信息
    except FileNotFoundError:
        return "No GPU found (nvidia-smi not found)" # 如果找不到 nvidia-smi 命令，设置提示信息


# 获取 CUDA 版本的函数
def get_cuda_version():
    try:
        # 尝试使用 nvcc --version 获取 CUDA 版本
        result = subprocess.run(['nvcc', '--version'], capture_output=True, text=True)
        if result.returncode == 0: # 如果命令成功执行
            for line in result.stdout.splitlines():
                if 'release' in line: # 查找包含 'release' 的行
                    return line.split('release ')[1].split(',')[0] # 解析行，提取版本号
        return "CUDA not found or version not parsed" # 如果找不到 CUDA 或版本无法解析，设置提示信息
    except FileNotFoundError:
        return "CUDA not found" # 如果找不到 nvcc 命令，设置提示信息

# 获取 Python 版本的函数
def get_python_version():
    return platform.python_version() # 获取 Python 版本

# 获取 Conda 版本的函数
def get_conda_version():
    try:
        # 尝试使用 conda --version 获取 Conda 版本
        result = subprocess.run(['conda', '--version'], capture_output=True, text=True)
        if result.returncode == 0: # 如果命令成功执行
            return result.stdout.strip() # 返回 Conda 版本
        return "Conda not found or version not parsed" # 如果找不到 Conda 或版本无法解析，设置提示信息
    except FileNotFoundError:
        return "Conda not found" # 如果找不到 conda 命令，设置提示信息

# 获取物理磁盘空间信息的函数
def get_disk_space():
    try:
        total, used, free = shutil.disk_usage("/") # 获取根目录的磁盘使用情况
        total_gb = total / (1024**3) # 转换为 GB
        used_gb = used / (1024**3) # 转换为 GB
        free_gb = free / (1024**3) # 转换为 GB
        return f"Total: {total_gb:.2f} GB, Used: {used_gb:.2f} GB, Free: {free_gb:.2f} GB" # 格式化输出
    except Exception as e:
        return f"Could not retrieve disk info: {e}" # 如果获取信息出错，设置错误信息

# 获取环境信息
os_name = platform.system() # 获取操作系统名称
os_version = platform.release() # 获取操作系统版本
if os_name == "Linux":
    try:
        # 在 Linux 上尝试获取发行版和版本
        lsb_info = subprocess.run(['lsb_release', '-a'], capture_output=True, text=True)
        if lsb_info.returncode == 0: # 如果命令成功执行
            for line in lsb_info.stdout.splitlines():
                if 'Description:' in line: # 查找包含 'Description:' 的行
                    os_version = line.split('Description:')[1].strip() # 提取描述信息作为版本
                    break # 找到后退出循环
                elif 'Release:' in line: # 查找包含 'Release:' 的行
                     os_version = line.split('Release:')[1].strip() # 提取版本号
                     # 尝试获取 codename
                     try:
                         codename_info = subprocess.run(['lsb_release', '-c'], capture_output=True, text=True)
                         if codename_info.returncode == 0:
                             os_version += f" ({codename_info.stdout.split(':')[1].strip()})" # 将 codename 添加到版本信息中
                     except:
                         pass # 如果获取 codename 失败则忽略

    except FileNotFoundError:
        pass # lsb_release 可能未安装，忽略错误

full_os_info = f"{os_name} {os_version}" # 组合完整的操作系统信息
cpu_info = get_cpu_info() # 调用函数获取 CPU 信息和核心数量
memory_info = get_memory_info() # 调用函数获取内存信息
gpu_info = get_gpu_info() # 调用函数获取 GPU 信息和显存
cuda_version = get_cuda_version() # 调用函数获取 CUDA 版本
python_version = get_python_version() # 调用函数获取 Python 版本
conda_version = get_conda_version() # 调用函数获取 Conda 版本
disk_info = get_disk_space() # 调用函数获取物理磁盘空间信息


# 创建用于存储数据的字典
env_data = {
    "项目": [ # 项目名称列表
        "操作系统",
        "CPU 信息",
        "内存信息",
        "GPU 信息",
        "CUDA 信息",
        "Python 版本",
        "Conda 版本",
        "物理磁盘空间" # 添加物理磁盘空间
    ],
    "信息": [ # 对应的信息列表
        full_os_info,
        cpu_info,
        memory_info,
        gpu_info,
        cuda_version,
        python_version,
        conda_version,
        disk_info # 添加物理磁盘空间信息
    ]
}

# 创建一个 pandas DataFrame
df = pd.DataFrame(env_data)

# 打印表格
print("### 环境信息") # 打印标题
print(df.to_markdown(index=False)) # 将 DataFrame 转换为 Markdown 格式并打印，不包含索引


# 智能助手（Assistants）

[智能助手](https://langchain-ai.github.io/langgraph/concepts/assistants/#resources) 为开发者提供了一种快速、结构化的方式来调整和版本化 LangGraph 智能体，便于持续迭代实验与测试。

## 为图（Graph）提供配置

我们的 `task_maistro` 图已经完成了接入智能助手所需的配置。

该图包含一个 `configuration.py` 文件，用于声明可以被助手覆盖的可配置字段，并在运行时自动加载。

在图的节点逻辑内部，我们可以通过 `user_id`、`todo_category`、`task_maistro_role` 等字段读取不同助手之间的差异。

## 创建智能助手

回到我们在前面章节中持续构建的 `task_maistro` 应用，它的最佳实践用例是什么？

对我而言，优势在于能够为不同场景维护独立的任务清单。

例如，我希望一个助手专注管理个人生活的待办事项，另一个助手负责工作任务。

这一目标可以通过调整 `todo_category` 与 `task_maistro_role` 这两个可配置字段轻松实现。

![智能助手配置界面截图](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/673d50597f4e9eae9abf4869_Screenshot%202024-11-19%20at%206.57.01%E2%80%AFPM.png)


In [ ]:
# 安装或升级 LangGraph SDK
# 使用 %%capture 魔法命令隐藏安装过程中的常规输出
# --no-stderr 参数确保出现错误时仍会显示在终端
%%capture --no-stderr
%pip install -U langgraph_sdk


这是部署图时自动创建的默认智能助手，可以作为之后定制助手的参照基线。


In [ ]:
# 导入 LangGraph SDK 客户端工厂
from langgraph_sdk import get_client

# 设置本地部署的 URL 地址
# 这里使用通过 CLI 启动的本地服务
url_for_cli_deployment = "http://localhost:8123"

# 创建客户端实例，用于与部署的图交互
client = get_client(url=url_for_cli_deployment)


### 个人助手

首先我们构建一个专门处理个人事务的智能助手。


In [ ]:
# 创建个人助手
# 调用客户端在部署的图上新建一个智能助手实例
personal_assistant = await client.assistants.create(
    # "task_maistro" 是该部署图的名称
    "task_maistro", 
    # 配置参数：待办类别设置为 "personal"（个人）
    config={"configurable": {"todo_category": "personal"}}
)

# 输出助手的关键信息，确认配置是否生效
print(personal_assistant)


让我们进一步完善这个助手，补充 `user_id` 等专属信息，并[创建一个新版本](https://langchain-ai.github.io/langgraph/cloud/how-tos/assistant_versioning/#create-a-new-version-for-your-assistant)。这样可以在不影响线上环境的前提下安全迭代配置。


In [ ]:
# 定义个人助手的角色设定
# 这段系统提示词决定了助手的语气与处理逻辑
task_maistro_role = """你是一位友好且条理清晰的个人任务助手，你的主要职责是帮助用户掌握个人任务和承诺的进度。具体要求：

- 协助跟踪并整理个人任务
- 当需要提供“待办事项摘要”时：
  1. 按截止时间（已逾期、今天、本周、未来）分组列出所有当前任务
  2. 标注任何缺少截止日期的任务，并温和地鼓励用户补充
  3. 提醒那些看起来重要却缺少预计耗时的任务
- 如果新增任务没有截止日期，要主动询问用户是否需要补充
- 在帮助用户保持自我管理的同时，始终保持支持性的语气
- 根据截止时间和重要性协助用户排列任务优先级

你的沟通风格应该积极、乐于助人，而不是评判。

当任务缺少截止日期时，可以这样回应：“我注意到 [任务] 还没有截止日期。要不要补充一个，帮助我们更好地追踪？”"""

# 组合需要更新的配置项
configurations = {"todo_category": "personal", 
                  "user_id": "lance",
                  "task_maistro_role": task_maistro_role}

# 更新个人助手配置
personal_assistant = await client.assistants.update(
    personal_assistant["assistant_id"],
    config={"configurable": configurations}
)

# 打印更新后的助手信息
print(personal_assistant)


### 工作助手

接下来再创建一个面向工作任务的助手，帮助我跟进团队安排。


In [ ]:
task_maistro_role = """你是一位专注且高效的工作任务助手。

你的核心职责是帮助用户以现实可行的时间表管理工作承诺。

具体要求：

- 协助跟踪并整理工作任务
- 当提供“待办事项摘要”时：
  1. 按截止时间（已逾期、今天、本周、未来）分组列出所有当前任务
  2. 标注任何缺少截止日期的任务，并温和地提醒用户补充
  3. 提醒那些看起来重要却缺少预计耗时的任务
- 与用户讨论新任务时，结合任务类型给出合理的时间建议：
  - 开发者关系相关功能：通常需要 1 天
  - 课程课件评审或反馈：通常需要 2 天
  - 文档冲刺：通常需要 3 天
- 根据截止日期和团队依赖关系帮助用户安排优先级
- 在协助用户履行承诺时保持专业、务实的语气

当任务缺少截止日期时，可以这样回应：“我注意到 [任务] 还没有截止日期。参考类似的任务，这类工作大约需要 [建议时长]。要不要据此设定一个截止日期？”"""

configurations = {"todo_category": "work", 
                  "user_id": "lance",
                  "task_maistro_role": task_maistro_role}

# 创建工作助手
# 使用客户端创建一个新的工作助手实例
work_assistant = await client.assistants.create(
    # "task_maistro" 是我们部署的图的名称
    "task_maistro", 
    # 配置参数：待办类别设置为 "work"（工作）
    config={"configurable": configurations}
)

# 打印创建的工作助手信息
print(work_assistant)


## 使用智能助手

智能助手的配置被持久化在部署所使用的 `Postgres` 数据库中。借助 SDK，我们可以方便地[检索](https://langchain-ai.github.io/langgraph/cloud/how-tos/configuration_cloud/)并管理这些助手。


In [ ]:
# 搜索所有智能助手
# 使用客户端搜索功能获取所有已创建的助手
assistants = await client.assistants.search()

# 遍历并打印每个助手的基本信息
for assistant in assistants:
    print({
        'assistant_id': assistant['assistant_id'],  # 助手的唯一标识符
        'version': assistant['version'],            # 助手的版本号
        'config': assistant['config']               # 助手的配置信息
    })

我们还可以通过 SDK 维护这些助手，例如删除不再使用的测试助手。

> 视频中的示例语法已经过时。下面的代码展示了更新后的做法：先创建一个临时助手，再将其删除。


In [ ]:
# 创建一个临时的智能助手
# 用于演示删除功能
temp_assistant = await client.assistants.create(
    "task_maistro", 
    config={"configurable": configurations}
)

# 搜索所有助手并显示删除前的状态
assistants = await client.assistants.search()
for assistant in assistants:
    print(f"删除前: {{'assistant_id': {assistant['assistant_id']}}}")
    
# 删除我们的临时助手
# 删除最后一个助手（刚创建的临时助手）
await client.assistants.delete(assistants[-1]["assistant_id"])
print()

# 再次搜索所有助手并显示删除后的状态
assistants = await client.assistants.search()
for assistant in assistants:
    print(f"删除后: {{'assistant_id': {assistant['assistant_id']} }}")

接下来，保存个人助手和工作助手的 ID，方便后续调用。


In [ ]:
# 设置助手ID变量，方便后续使用
# 从搜索结果中获取工作助手和个人助手的ID
work_assistant_id = assistants[0]['assistant_id']      # 工作助手的ID
personal_assistant_id = assistants[1]['assistant_id']  # 个人助手的ID

### 工作助手

先让工作助手处理几条任务，观察它的响应。


In [ ]:
# 导入必要的消息类型和转换工具
from langchain_core.messages import HumanMessage
from langchain_core.messages import convert_to_messages

# 用户输入：创建或更新几条待办事项
user_input = "请帮我创建或更新几条待办事项：1）今天下班前重新录制第 6 模块第 5 课。2）在下周一之前更新 audioUX。"

# 创建一个新的对话线程
# 线程用于管理用户与助手之间的完整对话历史
thread = await client.threads.create()

# 使用工作助手处理用户输入
# 通过流式处理持续获取助手的响应
async for chunk in client.runs.stream(thread["thread_id"], 
                                      work_assistant_id,
                                      input={"messages": [HumanMessage(content=user_input)]},
                                      stream_mode="values"):

    # 当接收到值事件时，处理并显示助手的响应
    if chunk.event == 'values':
        state = chunk.data
        # 将消息转换为可读格式并打印最后一条消息
        convert_to_messages(state["messages"])[-1].pretty_print()


In [ ]:
# 为工作助手新增一条待办
user_input = "再创建一条待办：完成报表生成教程的最终稿。"

# 新建对话线程，用于承载对话上下文
thread = await client.threads.create()

# 流式运行：将用户消息发送给工作助手，并逐步读取返回值
async for chunk in client.runs.stream(thread["thread_id"], 
                                      work_assistant_id,
                                      input={"messages": [HumanMessage(content=user_input)]},
                                      stream_mode="values"):

    # 当事件类型为 values 时，代表有新的状态返回
    if chunk.event == 'values':
        state = chunk.data
        # 打印助手的最后一条消息
        convert_to_messages(state["messages"])[-1].pretty_print()


助手会按照它的系统提示来约束输出，例如在创建任务时提醒我补充截止日期。


In [ ]:
# 回复助手：为该任务设置下周二为截止日期
user_input = "好的，这项任务就安排在下周二完成。"

# 继续在同一个线程中进行流式交互
async for chunk in client.runs.stream(thread["thread_id"], 
                                      work_assistant_id,
                                      input={"messages": [HumanMessage(content=user_input)]},
                                      stream_mode="values"):

    # 处理返回的状态值
    if chunk.event == 'values':
        state = chunk.data
        # 打印助手的最后一条消息
        convert_to_messages(state["messages"])[-1].pretty_print()


### 个人助手

同样的流程也适用于个人助手，我们可以为它创建或查询待办事项。


In [ ]:
# 为个人助手批量创建两条待办
user_input = "请创建两条待办：1）本周末了解宝宝游泳课的安排。2）为冬季出行确认 AmEx 积分。"

# 新建线程并以流式方式与个人助手交互
thread = await client.threads.create()
async for chunk in client.runs.stream(thread["thread_id"], 
                                      personal_assistant_id,
                                      input={"messages": [HumanMessage(content=user_input)]},
                                      stream_mode="values"):

    # 处理返回值事件
    if chunk.event == 'values':
        state = chunk.data
        # 打印助手的最后一条消息
        convert_to_messages(state["messages"])[-1].pretty_print()


In [ ]:
# 让个人助手给出当前待办摘要
user_input = "请给我一个待办事项摘要。"

# 新建线程并请求摘要
thread = await client.threads.create()
async for chunk in client.runs.stream(thread["thread_id"], 
                                      personal_assistant_id,
                                      input={"messages": [HumanMessage(content=user_input)]},
                                      stream_mode="values"):

    # 处理流式返回并打印最终消息
    if chunk.event == 'values':
        state = chunk.data
        convert_to_messages(state["messages"])[-1].pretty_print()
